In [1]:
import string
import glob
import nltk

# Copy all dialogs to one file

In [3]:
all_dialogs = ""
replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
for fname in glob.iglob('../data/clean_data/*/*.txt'):
    print fname
    prev_char = ''
    for line in open(fname):
        line = line.strip()
        parts = line.split(': ')
        character = parts[0]
        dialog = ' '.join(parts[1:])
        dialog = dialog.lower()
#         dialog = dialog.translate(replace_punctuation)
        if character != prev_char:
            all_dialogs += dialog + '\n'
        else:
            all_dialogs += dialog + ' '
        prev_char = character
with open('../res/all_dialogs.txt', 'w') as f:
    f.write(all_dialogs)

../data/clean_data/F/Fair Game.txt
../data/clean_data/F/Fast Times at Ridgemont High.txt
../data/clean_data/F/Five Easy Pieces.txt
../data/clean_data/F/Frozen (Disney).txt
../data/clean_data/F/Fantastic Four.txt
../data/clean_data/F/Field of Dreams.txt
../data/clean_data/F/Fabulous Baker Boys, The.txt
../data/clean_data/F/Frozen.txt
../data/clean_data/F/Flintstones, The.txt
../data/clean_data/F/Fargo.txt
../data/clean_data/F/Fatal Instinct.txt
../data/clean_data/F/Fright Night.txt
../data/clean_data/F/From Dusk Till Dawn.txt
../data/clean_data/F/Final Destination.txt
../data/clean_data/F/Friday the 13th.txt
../data/clean_data/F/Flight.txt
../data/clean_data/F/Flash Gordon.txt
../data/clean_data/F/Fantastic Mr Fox.txt
../data/clean_data/F/Freaked.txt
../data/clean_data/F/Forrest Gump.txt
../data/clean_data/F/Fracture.txt
../data/clean_data/F/Frances.txt
../data/clean_data/F/Family Man, The.txt
../data/clean_data/F/Ferris Bueller's Day Off.txt
../data/clean_data/F/Fear and Loathing in La

# Create vocab file

In [2]:
words = []
rejects = []
for line in open('../res/unigram_count.txt'):
    c, w = line.strip().split()
    if w.isalpha():
        words.append(w)
    else:
        rejects.append(w)
with open('../res/vocab.txt', 'w') as f:
    f.write('\n'.join(words))

with open('../res/rejects.txt', 'w') as f:
    f.write('\n'.join(rejects))

# Don't stem chars cos of chat problem

In [5]:
stems = set(line.strip() for line in open('../res/embed_vocab.txt'))
word_to_ind = {s.strip():int(i) for i, s in enumerate(open('../res/embed_vocab.txt'))}
our_stem_words = set(line.strip() for line in open('../res/vocab.txt'))

# Find the intersection between the two
new_stems = stems & our_stem_words

# Find the missing words
missing_stems = our_stem_words - stems

In [8]:
stems

{'lielisks',
 'boncet',
 '\xe3\x83\x97\xe3\x83\xaa\xe3\x83\xb3\xe3\x82\xbf\xe3\x83\xbc\xe3\x82\xa4\xe3\x83\xb3\xe3\x82\xaf',
 'hailam',
 'diperjuangin',
 'luanco',
 '\xd1\x83\xd1\x81\xd1\x82\xd1\x80\xd0\xbe\xd0\xb5\xd0\xbd\xd0\xbe',
 'bloqueos',
 'bloqueou',
 'lieliska',
 '\xd9\x84\xd9\x84\xd9\x85\xd9\x82\xd8\xa7\xd8\xb7\xd8\xb9\xd8\xa9',
 '\xef\xbd\x8e\xef\xbd\x88\xef\xbd\x8b\xef\xbd\x85\xe3\x83\x86\xe3\x83\xac\xef\xbc\x91\xe9\x95\xb7\xe9\x87\x8e',
 'spiders',
 '\xe7\x94\xb7\xe5\xad\x90\xe3\x81\xae\xe5\xa0\xb4\xe5\x90\x88\xe3\x81\xaf',
 'offendete',
 'boncel',
 'lieliski',
 'fsoo',
 'godaan',
 '\xd9\x84\xd9\x84\xd8\xa8\xd8\xad\xd9\x88\xd8\xab',
 'beliebersarehereforjustin',
 'parcheggi',
 'zannettik',
 '\xd8\xb3\xd9\x8f\xd8\xb1\xd9\x8f\xd8\xb1\xd9\x8d',
 'uahauahaua',
 'mempermudahnya',
 'zannettim',
 '\xed\x86\xb5\xed\x8c\x90\xed\x95\xa9\xeb\x8b\x88\xeb\x8b\xa4',
 '\xe5\x86\x86\xe5\xae\x89\xe3\x81\x84\xe6\x94\xbe\xe5\xb0\x84\xe8\x83\xbd\xe3\x81\xab\xe6\xb1\x9a\xe6\x9f\x93\xe3\x81\x95

In [4]:
new_stems = list(new_stems)
len(new_stems)

66532

In [12]:
inds = [word_to_ind[w] for w in new_stems]
new_stems = ['_PAD', '_GO', '_EOS', '_UNK'] + new_stems
with open('../res/filtered_vocab.txt', 'w') as f:
    f.write('\n'.join(new_stems))

In [13]:
import linecache
from tqdm import tqdm
embed_dims = 200
embed_file = '../data/glove.twitter.27B/glove.twitter.27B.' + str(embed_dims) + 'd.txt'
mat = []

# Add special words to embedding
special_words = ['_PAD', '_GO', '_EOS', '_UNK']
for i in range(len(special_words)):
    vector = [i]*embed_dims #size of embedding
    mat.append(vector)
    
for i in tqdm(inds):
    line = linecache.getline(embed_file, i)
    parts = line.strip().split()
    word = parts[0]
    vector = [float(f) for f in parts[1:]]
    mat.append(vector)

100%|██████████| 66532/66532 [00:38<00:00, 1717.61it/s] 


In [14]:
import numpy as np
embed = np.array(mat)
import cPickle as pickle
pickle.dump(embed, open('../res/embed_weights.pkl', 'w'))

In [15]:
embed.shape

(66536, 200)

In [16]:
from nltk.stem.porter import *
from tqdm import tqdm
stemmer = PorterStemmer()
nl = []
rejects = []
for line in tqdm(open('../res/all_dialogs.txt')):
    line = line.strip()
    s = ''
    for w in line.split():
        if not w.isalpha():
            w = "DIGIT"
#         else:
#             try:
#                 w = stemmer.stem(w)
#             except:
#                 rejects.append(w)
        s += w + ' '
    nl.append(s)

with open('../res/filtered_dialogs.txt', 'w') as f:
    f.write('\n'.join(nl))

747091it [00:04, 163573.10it/s]


In [17]:
lines = open('../res/filtered_dialogs.txt').readlines()

# Create train and validation files

In [24]:
even = lines[0::2]
odd = lines[1::2]

val_ind = int(len(even)*0.70)

test_ques = even[:-val_ind]
test_ans = odd[:-val_ind]
train_ques = even[-val_ind:]
train_ans = odd[-val_ind:]
with open('../res/train_ques.txt', 'w') as f:
    f.write(''.join(train_ques))
with open('../res/train_ans.txt', 'w') as f:
    f.write(''.join(train_ans))
with open('../res/val_ques.txt', 'w') as f:
    f.write(''.join(test_ques))
with open('../res/val_ans.txt', 'w') as f:
    f.write(''.join(test_ans))

In [28]:
val_ind = int(len(even)*0.70)

test_ques = even[:-val_ind]
test_ans = odd[:-val_ind]
train_ques = even[-val_ind:]
train_ans = odd[-val_ind:]

In [36]:
len(train_ques)

261482